In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

In [2]:
tf.enable_eager_execution()

In [3]:
tf.executing_eagerly()

True

In [26]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris

iris = load_iris()
_x = pd.DataFrame(iris.data[:, :], columns = iris.feature_names[:])
_y = pd.DataFrame(iris.target, columns =["Species"])
## Convert to binary
_y = _y[_y.Species != 2]
_x = _x[_y.Species != 2]

x_train, x_test, y_train, y_test = train_test_split(_x, _y, random_state=0)

In [27]:
y_train.shape

(75, 1)

In [13]:
# sw_t = tf.constant([[1.0, 2.0], [3.0, 4.0]])
# sw_t = tf.constant([[1.0, 1.0], [2.0, 1.0], [2.0, 1.0]])
sw_t = tf.constant([1.0, 1.0, 2.0, 1.0, 2.0, 1.0])

In [14]:
sw_t

<tf.Tensor: id=9, shape=(6,), dtype=float32, numpy=array([1., 1., 2., 1., 2., 1.], dtype=float32)>

In [ ]:
tf.matrix_inverse(sw_t)

In [6]:
tf.transpose(sw_t)

<tf.Tensor: id=3, shape=(2,), dtype=float32, numpy=array([1., 2.], dtype=float32)>

In [4]:
y_true = tf.constant([[1.0], [1.0], [1.0], [0.0], [0.0], [1.0], [0.0]])

In [9]:
print(y_true.get_shape())

(7, 1)


In [38]:
y_pred = tf.constant([[0.0], [1.0], [1.0], [1.0], [1.0], [0.0], [0.0]])

In [10]:
locations = tf.where (tf.equal (y_true, 1))

In [35]:
tf.where([False, True, False, True])

# locations

<tf.Tensor: id=606, shape=(2, 1), dtype=int64, numpy=
array([[1],
       [3]])>

In [22]:
tf.equal (y_true, 1)

<tf.Tensor: id=51, shape=(7, 1), dtype=bool, numpy=
array([[ True],
       [ True],
       [ True],
       [False],
       [False],
       [ True],
       [False]])>

In [15]:
indices = locations[:, 0]

In [30]:
y, idx = tf.unique (indices)

In [31]:
y

<tf.Tensor: id=215, shape=(4,), dtype=int64, numpy=array([0, 1, 2, 5])>

In [18]:
idx

<tf.Tensor: id=18, shape=(4,), dtype=int32, numpy=array([0, 1, 2, 3], dtype=int32)>

In [47]:
def fn(unique, indexes, preds):
    u_indexes = tf.where (tf.equal (unique, indexes))
    # print(tf.equal (unique, indexes))
    u_indexes = tf.reshape (u_indexes, (1, -1))

    X = tf.gather (preds, u_indexes)
    
    X_mean = X - tf.reduce_mean (X, axis=1)
    print('x_mean --', X_mean)
    m = tf.cast (tf.shape (X_mean)[1], tf.float32)
    print('m --', m)
    print('mat_mul --', tf.matmul (tf.transpose (X_mean[0]), X_mean[0]), '\n')
    # X_mean = tf.squeeze(X_mean)
    # print(tf.shape (X_mean))
    return (1 / (10 - 1)) * tf.matmul (tf.transpose (X_mean[0]), X_mean[0])

covs_t = tf.map_fn (lambda x: fn (x, indices, y_pred), y, dtype=tf.float32)[0]

x_mean -- tf.Tensor([[[0.]]], shape=(1, 1, 1), dtype=float32)
m -- tf.Tensor(1.0, shape=(), dtype=float32)
mat_mul -- tf.Tensor([[0.]], shape=(1, 1), dtype=float32) 

x_mean -- tf.Tensor([[[0.]]], shape=(1, 1, 1), dtype=float32)
m -- tf.Tensor(1.0, shape=(), dtype=float32)
mat_mul -- tf.Tensor([[0.]], shape=(1, 1), dtype=float32) 

x_mean -- tf.Tensor([[[0.]]], shape=(1, 1, 1), dtype=float32)
m -- tf.Tensor(1.0, shape=(), dtype=float32)
mat_mul -- tf.Tensor([[0.]], shape=(1, 1), dtype=float32) 

x_mean -- tf.Tensor([[[0.]]], shape=(1, 1, 1), dtype=float32)
m -- tf.Tensor(1.0, shape=(), dtype=float32)
mat_mul -- tf.Tensor([[0.]], shape=(1, 1), dtype=float32) 



In [58]:
tf.shape(y_pred)[0]

<tf.Tensor: id=2862, shape=(), dtype=int32, numpy=7>

In [48]:
covs_t

<tf.Tensor: id=2795, shape=(1, 1), dtype=float32, numpy=array([[0.]], dtype=float32)>

In [49]:
Sw_t = tf.reduce_mean (covs_t, axis=0)

In [50]:
Sw_t

<tf.Tensor: id=2798, shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>

In [51]:
Xt_bar = y_pred - tf.reduce_mean (y_pred, axis=0)
print('Xt_bar -- ', Xt_bar)
# m = T.cast(Xt_bar.shape[0], 'float32')
m = tf.cast (tf.shape (Xt_bar)[1], tf.float32)
# St_t = (1 / (m - 1)) * tf.matmul (tf.transpose (Xt_bar), Xt_bar)
St_t = (1 / (10 - 1)) * tf.matmul (tf.transpose (Xt_bar), Xt_bar)

Xt_bar --  tf.Tensor(
[[-0.5714286]
 [ 0.4285714]
 [ 0.4285714]
 [ 0.4285714]
 [ 0.4285714]
 [-0.5714286]
 [-0.5714286]], shape=(7, 1), dtype=float32)


In [53]:
Sb_t = St_t - Sw_t

print('Sb_t -- ', Sb_t)

# cope for numerical instability (regularize)
dim = tf.shape (y)[0]
Sw_t += tf.eye (dim) * 0.1 # 

Sb_t --  tf.Tensor([[0.19047621]], shape=(1, 1), dtype=float32)


In [54]:
Sw_t

<tf.Tensor: id=2847, shape=(4, 4), dtype=float32, numpy=
array([[0.1, 0. , 0. , 0. ],
       [0. , 0.1, 0. , 0. ],
       [0. , 0. , 0.1, 0. ],
       [0. , 0. , 0. , 0.1]], dtype=float32)>

In [62]:
th=33

evals_t = np.abs (tf.linalg.eigvalsh (tf.matrix_inverse (St_t) * Sw_t))
# evals_t = np.abs (tf.linalg.eigvalsh (St_t * Sw_t))

# compute eigenvalues
# evals_t = X = tf.matrix_solve(Sb_t, St_t) #Sb_t, St_t # B^{-1} A x = \lambda x
th_eigenvals = tf.contrib.distributions.percentile (evals_t, th)
evals_t_mask = evals_t > th_eigenvals
evals_t = tf.boolean_mask (evals_t, evals_t_mask)
cost = tf.reduce_mean (evals_t)

TypeError: unhashable type: 'list'

---

In [9]:
test_var = tf.constant([[4.0]])

In [11]:
tf.matrix_inverse (test_var)

<tf.Tensor: id=6, shape=(1, 1), dtype=float32, numpy=array([[0.25]], dtype=float32)>

In [10]:
test_var

<tf.Tensor: id=4, shape=(1, 1), dtype=float32, numpy=array([[4.]], dtype=float32)>